In [0]:
from datetime import datetime
import requests
import os
from pyspark.sql import SparkSession

In [0]:
# =====================================
# CONFIGURAÇÕES
# =====================================
spark = SparkSession.builder.getOrCreate()

# Data de referência = hoje
data_ref = datetime.now().strftime("%Y%m%d")

# URL pública do CSV diário
url = f"https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/focos_diario_br_{data_ref}.csv"

# Caminho destino no volume Databricks
destino = f"/Volumes/datamasters/raw/raw_inpe/focos_diario_br_{data_ref}.csv"

print(f"Baixando arquivo: {url}")
print(f"Gravando em: {destino}")

In [0]:
# =====================================
# DOWNLOAD DO ARQUIVO
# =====================================
try:
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Falha ao baixar arquivo ({response.status_code}): {url}")
    
    # Escreve no diretório do volume (Databricks Volumes já montado)
    with open(destino, "wb") as f:
        f.write(response.content)
    
    print(f"Arquivo {destino} gravado com sucesso ✅")

except Exception as e:
    raise RuntimeError(f"Erro no download do arquivo INPE: {e}")

In [0]:
# =====================================
#  LEITURA PARA VERIFICAÇÃO
# =====================================
df = spark.read.option("header", True).option("delimiter", ",").csv(destino)
print(f"Arquivo contém {df.count()} registros.")